In [1]:
import sys
sys.path.append('./minGPT')
from mingpt.model import GPT

import torch
import torch.nn as nn
from torch.nn import TransformerDecoder, TransformerDecoderLayer
from torch.utils.data import Dataset, DataLoader
import numpy as np
from sklearn.model_selection import train_test_split
import re

# from mingpt.model import GPT
 
# model_config = GPT.get_default_config()
# model_config.vocab_size = len(vocab)  # 词汇表大小
# model_config.block_size = 20          # 序列最大长度
# model_config.n_layer = 6              # 参考网页6的层数配置
# model_config.n_head = 8
# model = GPT(model_config)


In [2]:
import os
print(os.getcwd())

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if device.type == 'cpu':
    file_path = '../kaggleData/data/eng-cmn.txt'
else:
    file_path = '/kaggle/input/eng-cmn/eng-cmn.txt'

d:\GITrepo\DL-MDS5122-hw2\mingpt


In [3]:
text_pairs = []

with open(file_path, 'r', encoding='utf-8') as f:
    for line in f:

        parts = line.strip().split('\t')
        

        if len(parts) < 3:
            continue
            
        en_text = parts[0].strip()
        zh_text = parts[1].strip()
        

        if en_text and zh_text:

            text_pairs.append((zh_text, en_text))

In [6]:
from collections import defaultdict
import re
# def build_vocab(text_pairs):
#     vocab = defaultdict(lambda: len(vocab))
#     special_tokens = ["<pad>", "<sos>", "<eos>", "<sep>"]
#     for token in special_tokens:
#         vocab[token]
    
#     for ch, en in text_pairs:
#         for char in (ch + en):
#             vocab[char]
#     return vocab
def build_vocab(text_pairs):
    vocab = defaultdict(lambda: len(vocab))
    special_tokens = ["<pad>", "<sos>", "<eos>", "<sep>"]
    for token in special_tokens:
        vocab[token]
    
    for ch, en in text_pairs:
        ch = re.sub(r'[^\w\s\u4e00-\u9fa5]', '', ch, flags=re.UNICODE)
        for char in ch.lower():
            vocab[char]
        en = re.sub(r'[^\w\s\u4e00-\u9fa5]', '', en, flags=re.UNICODE)
        for char in en.lower().split(' '):
            vocab[char]
    return vocab

def text_to_ids_cn(text, vocab, add_special_tokens=False):
    text = re.sub(r'[^\w\s\u4e00-\u9fa5]', '', text, flags=re.UNICODE)
    text = re.sub(r'[^\w\s\u4e00-\u9fa5]', '', text, flags=re.UNICODE)
    text=text.lower()
    ids = []
    if add_special_tokens:
        ids.append(vocab["<sos>"])
    for char in text:
        ids.append(vocab[char])
    # ids += [10, 2476, 2477, 85]
    if add_special_tokens:
        ids.append(vocab["<eos>"])
    return ids

def text_to_ids_eng(text, vocab, add_special_tokens=False):
    text = re.sub(r'[^\w\s\u4e00-\u9fa5]', '', text, flags=re.UNICODE)
    text = re.sub(r'[^\w\s\u4e00-\u9fa5]', '', text, flags=re.UNICODE)
    text=text.lower()
    ids = []
    if add_special_tokens:
        ids.append(vocab["<sos>"])
    for char in text.split(' '):
        ids.append(vocab[char])
    
    if add_special_tokens:
        ids.append(vocab["<eos>"])
    return ids


from torch.utils.data import Dataset, DataLoader

class TranslationDataset(Dataset):
    def __init__(self, pairs, vocab, max_len=50):
        self.pairs = pairs
        self.vocab = vocab
        self.max_len = max_len

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, idx):
        ch, en = self.pairs[idx]

        input_ids = (
            [self.vocab["<sos>"]] +
            text_to_ids_cn(ch, self.vocab) +
            [self.vocab["<sep>"]] +
            text_to_ids_eng(en, self.vocab) +
            [self.vocab["<eos>"]]
        )

        input_ids = input_ids[:self.max_len]
        src = input_ids[:-1]
        tgt = input_ids[1:]
        # input_ids += [self.vocab["<pad>"]] * (self.max_len - len(input_ids))
        src += [self.vocab["<pad>"]]* (self.max_len - len(src))
        tgt += [self.vocab["<pad>"]]* (self.max_len - len(tgt))
        # return torch.tensor(input_ids[:-1]), torch.tensor(input_ids[1:]) 
        return torch.tensor(src), torch.tensor(tgt)
    

vocab = build_vocab(text_pairs)
    
vocab_size = len(vocab)
eval_num = 100 # translate eval_num of inputs
BATCH_SIZE = 32
MAX_LEN = 49
model_max_seq=128
max_epoch = 30
num_heads = 2


dataset = TranslationDataset(text_pairs, vocab, MAX_LEN)
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

In [20]:
from mingpt.model import GPT
from mingpt.trainer import Trainer

model_config = GPT.get_default_config()
model_config.model_type = 'gpt-mini'
model_config.vocab_size = len(vocab) # openai's model vocabulary
model_config.block_size = 100  # openai's model block_size (i.e. input context length)
model = GPT(model_config)

# train config
train_config = Trainer.get_default_config()
train_config.learning_rate = 5e-4      
train_config.batch_size = 32
train_config.max_iters = 5000          
train_config.num_workers = 0

# start training
trainer = Trainer(train_config, model, dataset)
# trainer.run()

def batch_end_callback(trainer):
    if trainer.iter_num % 100 == 0:
        print(f"iter_dt {trainer.iter_dt * 1000:.2f}ms; iter {trainer.iter_num}: train loss {trainer.loss.item():.5f}")
trainer.set_callback('on_batch_end', batch_end_callback)

trainer.run()

number of parameters: 4.78M
running on device cpu
iter_dt 0.00ms; iter 0: train loss 9.39832
iter_dt 295.72ms; iter 100: train loss 2.29008
iter_dt 201.28ms; iter 200: train loss 1.56079
iter_dt 246.24ms; iter 300: train loss 1.70178
iter_dt 249.06ms; iter 400: train loss 1.96999
iter_dt 242.18ms; iter 500: train loss 1.76812
iter_dt 245.91ms; iter 600: train loss 1.61267
iter_dt 228.36ms; iter 700: train loss 1.66291
iter_dt 239.50ms; iter 800: train loss 1.69228
iter_dt 252.66ms; iter 900: train loss 1.44959
iter_dt 238.86ms; iter 1000: train loss 1.63701
iter_dt 252.69ms; iter 1100: train loss 1.57210
iter_dt 251.72ms; iter 1200: train loss 1.31319
iter_dt 219.13ms; iter 1300: train loss 1.33743
iter_dt 241.27ms; iter 1400: train loss 1.34280
iter_dt 220.41ms; iter 1500: train loss 1.45900
iter_dt 267.05ms; iter 1600: train loss 1.43678
iter_dt 249.22ms; iter 1700: train loss 1.30029
iter_dt 235.19ms; iter 1800: train loss 1.18036
iter_dt 270.38ms; iter 1900: train loss 1.28492
iter

In [23]:
# weights save and reload
torch.save(model.state_dict(), 'mingpt.pth') # about 26MB
model.load_state_dict(torch.load('mingpt.pth'))

<All keys matched successfully>

In [21]:
id2w = {v:k for k, v in vocab.items()}
def decode(ids, id2w=id2w):
    tokens = [id2w.get(t, 'notfound') for t in ids if t !=2]
    return ' '.join(tokens)

In [25]:
import random
model.eval()
for i in range(20):
    with torch.no_grad():
        pair = random.choice(text_pairs)
        src = pair[0]
        tgt = pair[1]
        input_ids = (
            [vocab["<sos>"]] +
            text_to_ids_cn(src, vocab) +
            [vocab["<sep>"]]
        )
        input_tensor = torch.tensor([input_ids], device=device)
        print(f'用户输入: {src}')
        print(f'正确翻译: {tgt}')

        max_new_token = 20
        
        gen = model.generate(input_tensor,  max_new_tokens=max_new_token, do_sample=False, top_k=40)
        print('模型翻译：',decode(gen[0][-max_new_token:].tolist()))
        print('='*20)

用户输入: 不要再让这种事发生了！
正确翻译: Don't let that happen again.
模型翻译： dont let this happen again
用户输入: 你收到我寄給你的包裹了嗎？
正确翻译: Did you get the package that I sent you?
模型翻译： did you hear my bag
用户输入: 你在私立高中读书吗？
正确翻译: Are you a student of a private high school?
模型翻译： are you reading a doctor
用户输入: 我給了瑪麗一本書。
正确翻译: I gave Mary a book.
模型翻译： i gave mary a book
用户输入: 去度假的时候我要租辆车。
正确翻译: When I go on vacation, I'll rent a car.
模型翻译： i want to get a taxi
用户输入: 聞起來好香喔。你在煮什麼？
正确翻译: This smells great! What are you cooking?
模型翻译： the price of you look at the bottle
用户输入: 湯姆對我不友善。
正确翻译: Tom wasn't nice to me.
模型翻译： tom is not interested in love
用户输入: 告訴我你的故事。 我會注意聽。
正确翻译: Tell me your story. I am all ears.
模型翻译： tell me the story ill hear about it
用户输入: 请给我看看另一个。
正确翻译: Please show me another one.
模型翻译： please show me one
用户输入: 汤姆不知道玛丽是个连环杀手。
正确翻译: Tom had no idea that Mary was a serial killer.
模型翻译： tom doesnt know mary were a watch
用户输入: 老師跟他們說，叫他們別在河裡游泳。
正确翻译: The teacher told them not to swim in the river.
模型翻